In [12]:
install.packages('units')

Warning message in install.packages("units"):
“installation of package ‘units’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [8]:
install.packages('tigris', dependencies = TRUE)

also installing the dependencies ‘terra’, ‘units’, ‘raster’, ‘sf’, ‘leaflet’, ‘tidycensus’


Warning message in install.packages("tigris", dependencies = TRUE):
“installation of package ‘terra’ had non-zero exit status”
Warning message in install.packages("tigris", dependencies = TRUE):
“installation of package ‘units’ had non-zero exit status”
Warning message in install.packages("tigris", dependencies = TRUE):
“installation of package ‘raster’ had non-zero exit status”
Warning message in install.packages("tigris", dependencies = TRUE):
“installation of package ‘sf’ had non-zero exit status”
Warning message in install.packages("tigris", dependencies = TRUE):
“installation of package ‘leaflet’ had non-zero exit status”
Warning message in install.packages("tigris", dependencies = TRUE):
“installation of package ‘tigris’ had non-zero exit status”
Warning message in install.packages("tigris", dependencies = TRUE):
“installation of package ‘tidycensus’ had non-zero exit status”
Updating HTM

In [10]:
install.packages("devtools")
devtools::install_github('walkerke/tigris')

also installing the dependencies ‘credentials’, ‘gitcreds’, ‘ini’, ‘curl’, ‘gert’, ‘gh’, ‘whisker’, ‘downlit’, ‘httr2’, ‘rmarkdown’, ‘xopen’, ‘brew’, ‘usethis’, ‘pkgbuild’, ‘pkgdown’, ‘profvis’, ‘rcmdcheck’, ‘remotes’, ‘roxygen2’, ‘rversions’, ‘sessioninfo’, ‘urlchecker’


Warning message in install.packages("devtools"):
“installation of package ‘gert’ had non-zero exit status”
Warning message in install.packages("devtools"):
“installation of package ‘usethis’ had non-zero exit status”
Warning message in install.packages("devtools"):
“installation of package ‘devtools’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



ERROR: Error in loadNamespace(x): there is no package called ‘devtools’


In [2]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. NC Well Metals/P1011.2. Analyses/P1011.2.4. Contamination Mapping/Output')
cur_date = "010925"

library(readxl)
library(tidyverse)
library(reshape2)
library(tigris)
library(cowplot)
# library(rcartocolor)
# library(RColorBrewer)
# library(MetBrewer)
# library(tidytext)

# reading in files
as_census_tracts_df = data.frame(read_excel("Input/As_Census_Tracts_010925.xlsx"))
mn_census_tracts_df = data.frame(read_excel("Input/Mn_Census_Tracts_010925.xlsx"))

as_census_tracts_df$Concentration = as.numeric(as_census_tracts_df$Concentration)
mn_census_tracts_df$Concentration = as.numeric(mn_census_tracts_df$Concentration)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths



Attaching package: ‘cowplot’


The following object is masked from ‘package:lubridate’:

    stamp


Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [3]:
head(as_census_tracts_df)
head(mn_census_tracts_df)

,GEOID,Concentration
,<dbl>,<dbl>
1,37179020902,14.168383
2,37179020901,13.287818
3,37179020100,18.720882
4,37179020800,16.450622
5,37179020401,1.731546
6,37179020202,27.408738


,GEOID,Concentration
,<dbl>,<dbl>
1,37179020902,147.44876
2,37179020901,202.77838
3,37179020100,171.13772
4,37179020800,620.25841
5,37179020401,75.00000
6,37179020202,28.25828


ADD WORDS

In [4]:
# download shapefile from the tigris (census package)
nc_census_tract_shapefile = tracts(state = 37, year = 2018) %>%
    rename(FIPS = GEOID10)

head(nc_census_tract_shapefile)

ERROR: Error in tracts(state = 37, year = 2018): could not find function "tracts"
